# bootstrap_puzzles_02_update_word_states

Using word data from past puzzles, updates the initial `silver.word_states` table based on the `bronze.word_decisions` records created after ingesting all of the past puzzles up to (including) 2025-06-23.

This notebook performs the update all in one shot, rather than month by month. If this causes memory-related performance issues, we can refactor to run month by month. Another TODO will be to have the pipeline for the previous stage (which goes year/month by year/month) loop through all of the past years (2023, 2024, 2025) and run the notebook for each one, before proceeding to this stage.

The `silver.word_states` table reflects the latest state of each word we are tracking, specifically:

- `last_seen_on`: the date of the most recent explicit or implicit decision made about the word (can be null if the word has never been a possible puzzle answer)
- `label`:
    - `1.0` if the most recent decision explicitly included in the word in the official solution
    - `0.0` if the most recent decision implicity rejected the word (it could be formed with the puzzle letters but was not included in the official solution)
    - `null` if the word has never been a possible puzzle answer
- `batch_id`: an identifier of the current puzzle or date range being processed, to support idempotent ops and redos
- the `word`, `letter_set`, `frequency`, `embedding` columns from `bronze.words` for the given word.

The process of updating the `silver.word_states` table is:

- Read `bronze.word_decisions` into a data frame and use a window function to select the most recent decision about each word in the table. (For each word, find the decision with the latest `puzzle_date`.)
- Rename `puzzle_date` to `last_seen_on`
- Rename `accepted` to `label`
- Add `batch_id` `"bootstrap_puzzles_1"` for this notebook
- Identify new words and query/join their `letter_set`, `embedding`, and `frequency` columns so they can be inserted
- Use Delta `MERGE INTO` semantics to update the latest decisions:
    - merge into on the key match `source.word = target.word`
    - `whenMatchedUpdate` on the condition `target.last_seen_on IS NULL OR source.last_seen_on >= target.last_seen_on`
        - update the `last_seen_on`, `label`, and `batch_id` columns with the value in the source
        - leave the `word`, `letter_set`, `frequency`, and `embedding` columns unchanged in the target (these are static properties of the word, not the decision). 
    - `whenNotMatchedInsert` for decisions about words that are not present in the target table (this is not expected in bootstrap)
    - `whenNotMatchedBySourceDelete` to handle the case of reruns that must clean up rows from a previous run that should not have been added. The condition is `source.batch_id = target.batch_id`



In [ ]:
%run "./00_setup.ipynb"

In [ ]:
# Config for this notebook, possibly local only
# TODO: Remove this if not needed except for debugging

from src.envutils import is_databricks_env

if not is_databricks_env():
    print("updating spark config for this notebook")
    spark.conf.set("spark.sql.parquet.columnarReaderBatchSize", "1024")

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from delta.tables import DeltaTable

In [ ]:
# TODO: parameterize
_SOURCE_DB_NAME = "bronze"
_SOURCE_DECISIONS_TABLE_NAME = "word_decisions"
_SOURCE_WORDS_TABLE_NAME = "words"
_TARGET_DB_NAME = "silver"
_TARGET_TABLE_NAME = "word_states"

In [ ]:
# Get the word decisions
df = spark.sql(f"SELECT * FROM {_SOURCE_DB_NAME}.{_SOURCE_DECISIONS_TABLE_NAME}")
print(f"{df.count()} word_decisions in {_SOURCE_DB_NAME}.{_SOURCE_DECISIONS_TABLE_NAME}")

In [ ]:
num_distinct = df.select("word").distinct().count()
print(f"Number of distinct words in {_SOURCE_DB_NAME}.{_SOURCE_DECISIONS_TABLE_NAME}: {num_distinct}")

In [ ]:
# Define window partitioned by word, ordered by puzzle_date
window_spec = Window.partitionBy("word").orderBy(F.col("puzzle_date").desc())

In [ ]:
# Add row number to identify the most recent record
# Then filter on the first (most recent record)
filtered_df = df.withColumn("rn", F.row_number().over(window_spec)) \
                .filter(F.col("rn") == 1) \
                .drop("rn")

print(f"Num records after filter: {filtered_df.count()}")

In [ ]:
# Rename puzzle_date to last_seen_on
# Add batch_id "bootstrap_puzzles_1" for this notebook
BATCH_ID = "bootstrap_puzzles_1"
source_df = filtered_df.withColumnRenamed("puzzle_date", "last_seen_on") \
                       .withColumnRenamed("accepted", "label") \
                       .withColumn("batch_id", F.lit(BATCH_ID))

In [ ]:
# Identify which words in source_df won't be matched in the target table
target_words_df = spark.sql(f"SELECT word FROM {_TARGET_DB_NAME}.{_TARGET_TABLE_NAME}")
target_words = set([row.word for row in target_words_df.select("word").collect()])

source_words = set([row.word for row in source_df.select("word").collect()])

new_words = source_words - target_words
print(f"Found {len(target_words)} distinct words in target.")
print(f"Found {len(source_words)} distinct words in source.")
print(f"Found {len(new_words)} new words in source.")
if len(new_words) > 0:
    print(f"New words: {', '.join(sorted(new_words))}")

In [ ]:
if len(new_words) > 0:
    # Get embeddings and frequencies for new words, join to source_df
    formatted_new_words = ', '.join([f"'{word}'" for word in sorted(new_words)])
    query = f"""
        SELECT word, letter_set, embedding, frequency
        FROM {_SOURCE_DB_NAME}.{_SOURCE_WORDS_TABLE_NAME}
        WHERE word IN ({formatted_new_words})
    """
    new_words_df = spark.sql(query)
    
    # Join to enrich source_df with extra columns
    source_df = source_df.join(new_words_df, on="word", how="left")
else:
    # No new words. Add NULL columns explicitly
    source_df = source_df.withColumn("letter_set", F.lit(None).cast("string")) \
                         .withColumn("embedding", F.lit(None).cast("array<float>")) \
                         .withColumn("frequency", F.lit(None).cast("float"))

In [ ]:
target_table = DeltaTable.forName(spark, f"{_TARGET_DB_NAME}.{_TARGET_TABLE_NAME}")

In [ ]:
# Use DeltaMergeBuilder for the merge operation
merge_builder = target_table.alias("target").merge(
    source_df.alias("source"),
    "target.word = source.word"
)

# Define the merge logic
merge_builder.whenMatchedUpdate(
    condition="target.last_seen_on is null OR source.last_seen_on >= target.last_seen_on",
    set={
        "label": "source.label",
        "last_seen_on": "source.last_seen_on",
        "batch_id": "source.batch_id"
    }
).whenNotMatchedInsert(
    values={
        "word": "source.word",
        "letter_set": "source.letter_set",
        "frequency": "source.frequency",
        "embedding": "source.embedding",
        "last_seen_on": "source.last_seen_on", 
        "label": "source.label",
        "batch_id": "source.batch_id"
    }
).whenNotMatchedBySourceDelete(
    condition=f"target.batch_id = '{BATCH_ID}'"
).execute()
